In [1]:
import pandas as pd
import numpy as np
from scipy import signal
from tqdm import tqdm
from numpy.fft import fft, fftshift
import random
import warnings
warnings.filterwarnings(action='ignore')

In [24]:
import os
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from numpy.random import seed

In [3]:
# data load
# path 변수를 적절히 변경
x_train_path = os.path.join('data/train_features.csv')
y_train_path = os.path.join('data/train_labels.csv')
x_test_path = os.path.join('data/test_features.csv')
sub_path = os.path.join('data/sample_submission.csv')

train = pd.read_csv(x_train_path)
train_label = pd.read_csv(y_train_path)
test = pd.read_csv(x_test_path)
sub = pd.read_csv(sub_path)

In [4]:
train['acc_Energy']=(train['acc_x']**2+train['acc_y']**2+train['acc_z']**2)**(1/3)
test['acc_Energy']=(test['acc_x']**2+test['acc_y']**2+test['acc_z']**2)**(1/3)

train['gy_Energy']=(train['gy_x']**2+train['gy_y']**2+train['gy_z']**2)**(1/3)
test['gy_Energy']=(test['gy_x']**2+test['gy_y']**2+test['gy_z']**2)**(1/3)

train['gy_acc_Energy']=((train['gy_x']-train['acc_x'])**2+(train['gy_y']-train['acc_y'])**2+(train['gy_z']-train['acc_z'])**2)**(1/3)
test['gy_acc_Energy']=((test['gy_x']-test['acc_x'])**2+(test['gy_y']-test['acc_y'])**2+(test['gy_z']-test['acc_z'])**2)**(1/3)

In [5]:
dt=0.02 
def jerk_signal(signal): 
        return np.array([(signal[i+1]-signal[i])/dt for i in range(len(signal)-1)])

In [6]:
train_dt=[]
for i in tqdm(train['id'].unique()):
    temp=train.loc[train['id']==i]
    for v in train.columns[2:]:
        values=jerk_signal(temp[v].values)
        values=np.insert(values,0,0)
        temp.loc[:,v+'_dt']=values
    train_dt.append(temp)

100%|██████████████████████████████████████| 3125/3125 [00:26<00:00, 116.34it/s]


In [7]:
test_dt=[]
for i in tqdm(test['id'].unique()):
    temp=test.loc[test['id']==i]
    for v in train.columns[2:]:make_dataset(test_sc)
        values=jerk_signal(temp[v].values)
        values=np.insert(values,0,0)
        temp.loc[:,v+'_dt']=values
    test_dt.append(temp)

100%|████████████████████████████████████████| 782/782 [00:06<00:00, 126.89it/s]


In [8]:
from scipy import fftpack
from numpy.fft import *

def fourier_transform_one_signal(t_signal):
    complex_f_signal= fftpack.fft(t_signal)
    amplitude_f_signal=np.abs(complex_f_signal)
    return amplitude_f_signal

In [9]:
train=pd.concat(train_dt)

In [10]:
fft=[]
for i in tqdm(train['id'].unique()):
    temp=train.loc[train['id']==i]
    for i in train.columns[2:8]:
        temp[i]=fourier_transform_one_signal(temp[i].values)
    fft.append(temp)
train=pd.concat(fft)

100%|██████████████████████████████████████| 3125/3125 [00:06<00:00, 512.90it/s]


In [11]:
test=pd.concat(test_dt)

In [12]:
fft_t=[]
for i in tqdm(test['id'].unique()):
    temp=test.loc[test['id']==i]
    for i in test.columns[2:8]:
        temp[i]=fourier_transform_one_signal(temp[i].values)
    fft_t.append(temp)
test=pd.concat(fft_t)

100%|████████████████████████████████████████| 782/782 [00:00<00:00, 872.53it/s]


In [13]:
col=train.columns
train_s=train.copy()
test_s=test.copy()

In [14]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler

scaler = StandardScaler()

train_s.iloc[:,2:]= scaler.fit_transform(train_s.iloc[:,2:])
train_sc = pd.DataFrame(data = train_s,columns =col)

test_s.iloc[:,2:]= scaler.transform(test_s.iloc[:,2:])
test_sc = pd.DataFrame(data = test_s,columns =col)

In [17]:
series_train = train_sc.iloc[:, 2:].to_numpy().reshape(-1, 600, 18)
series_test = test_sc.iloc[:, 2:].to_numpy().reshape(-1,600, 18)

In [18]:
series_train.shape, series_test.shape

((3125, 600, 18), (782, 600, 18))

In [41]:
# label one-hot 인코딩
labels = train_label['label'].to_numpy()

In [42]:
import tensorflow as tf

In [43]:
# dataset과 validation set을 만들어 주는 함수
# validation set은 shuffle 적용 x
def make_train(series_data, labels):
  cat_y = tf.keras.utils.to_categorical(labels)

  BATCH_SIZE = 64
  train_dataset = tf.data.Dataset.from_tensor_slices((series_data, cat_y))
  train_dataset = train_dataset.batch(BATCH_SIZE)
  train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

  return train_dataset

def make_val(series_data, labels):
  cat_y = tf.keras.utils.to_categorical(labels)

  BATCH_SIZE = 64
  val_dataset = tf.data.Dataset.from_tensor_slices((series_data, cat_y))
  val_dataset = val_dataset.batch(BATCH_SIZE)
  val_dataset = val_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

  return train_dataset

In [44]:
from tensorflow import keras
from tensorflow.keras import backend as K

In [45]:
# 가중치 적용 loss
# https://openaccess.thecvf.com/content_CVPR_2019/papers/Cui_Class-Balanced_Loss_Based_on_Effective_Number_of_Samples_CVPR_2019_paper.pdf
# 위 논문 참고
def weighted_loss(labels, y_pred, beta=0.9):
        no_samples = [12, 21, 20, 23, 35, 25, 24, 26, 97, 37, 20, 23, 12,
                     12, 25, 25, 22, 27, 47, 20, 26, 27, 19, 20, 35, 24,
                     1518, 34, 55, 20, 35, 20, 18, 20, 22, 30, 28, 35, 20,
                     20, 34, 20, 20, 35, 21, 22, 20, 26, 25, 30, 37, 24,
                     12, 13, 23, 37, 36, 20, 20, 23, 48]

        weight = tf.divide(1-beta, 1-tf.pow(beta, no_samples))
        #y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        loss = labels * K.log(y_pred) * weight
        loss = -K.sum(loss, -1)
        return loss

def focal(labels, y_pred, gamma=2):
    #y_pred = tf.nn.softmax(logits, dim=-1)  # [batch_size, num_classes]
    #labels = tf.squeeze(labels)  # label example: [0,1,2,3]
    #labels = tf.to_float(tf.one_hot(labels, depth=y_pred.shape[1]))

    loss = -labels * ((1 - y_pred) ** gamma) * K.log(y_pred)
    loss = tf.reduce_mean(tf.reduce_sum(loss, axis=1))
    return loss


# 모델을 만들어 주는 함수
def base():
    seed(2021)
    tf.random.set_seed(2021)
    model = keras.models.Sequential([
            keras.layers.Conv1D(filters=128, kernel_size=9, padding='same', input_shape=[600, 18]),
            keras.layers.BatchNormalization(),
            keras.layers.Activation('relu'),
            keras.layers.Dropout(0.3),
            keras.layers.Conv1D(filters=256, kernel_size=6, padding='same'),
            keras.layers.BatchNormalization(),
            keras.layers.Activation('relu'),
            keras.layers.Dropout(0.4),
            keras.layers.Conv1D(filters=128, kernel_size=3,padding='same'),
            keras.layers.BatchNormalization(),
            keras.layers.Activation('relu'),
            keras.layers.Dropout(0.5),
            keras.layers.GlobalAveragePooling1D(),
            keras.layers.Dense(61, activation='softmax')
    ])
    model.compile(optimizer='adam',
                loss='categorical_crossentropy', 
                metrics=['accuracy'])
    return model

In [59]:
beta = 0.9
no_samples = [12, 21, 20, 23, 35, 25, 24, 26, 97, 37, 20, 23, 12,
                     12, 25, 25, 22, 27, 47, 20, 26, 27, 19, 20, 35, 24,
                     1518, 34, 55, 20, 35, 20, 18, 20, 22, 30, 28, 35, 20,
                     20, 34, 20, 20, 35, 21, 22, 20, 26, 25, 30, 37, 24,
                     12, 13, 23, 37, 36, 20, 20, 23, 48]
weight = (1.0-beta) / (1.0-np.power(beta, no_samples))
weight = MinMaxScaler(feature_range=(1, 2)).fit_transform(weight.reshape(-1, 1))

In [47]:
# checkpoint path
# 중간중간 모델의 weight를 저장할 경로 설정
ckpt_name = 'fold_cnn_weighted_ckpt.hdf5'
checkpoint_dir_path = os.path.join('checkpoint')
checkpoint_path = os.path.join(checkpoint_dir_path, ckpt_name)

# callback 함수 목록
callbacks_list = [
    # 매 epoch 마다 val_loss를 체크하여 가장 낮은 상태의 weight를 저장
    tf.keras.callbacks.ModelCheckpoint(
        filepath = checkpoint_path,
        monitor='val_loss',
        mode='min',
        save_weights_only=True,test_wt.csv
        save_best_only=True
    ),
    # 8번 동안 val_loss의 향상이 없으면 훈련 종료
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        mode='min',
        verbose=1, 
        patience=8
    ),
    tf.keras.callbacks.ReduceLROnPlateau(patience = 4,verbose = 1,factor = 0.5) 
]

In [48]:
from sklearn.model_selection import StratifiedKFold

In [28]:
no_samples = [12, 21, 20, 23, 35, 25, 24, 26, 97, 37, 20, 23, 12,
                     12, 25, 25, 22, 27, 47, 20, 26, 27, 19, 20, 35, 24,
                     1518, 34, 55, 20, 35, 20, 18, 20, 22, 30, 28, 35, 20,
                     20, 34, 20, 20, 35, 21, 22, 20, 26, 25, 30, 37, 24,
                     12, 13, 23, 37, 36, 20, 20, 23, 48]

beta = 0.9
weight = (1.0-beta) / (1.0-np.power(beta, no_samples))

In [61]:
weight = {i: w for i, w in enumerate(weight)}

In [58]:
# k-fold 적용
# 데이터 수가 적어 10개의 fold 사용
k = 10 
split = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
results = []    # 매 fold 훈련 후 loss 저장
models = []     # 매 fold 훈련 후 해당 모델 저장
accs = []
for i, (train, val) in enumerate(split.split(series_train, labels)):
    print('Fold', i)
    print('#'*20)

    train_dataset = make_train(series_train[train], labels[train])
    val_dataset = make_val(series_train[val], labels[val])

    model = base()

    model.fit(train_dataset, validation_data=val_dataset, callbacks=callbacks_list, 
              class_weight=weight, epochs=100)

    # 모델 복원
    model.load_weights(checkpoint_path)
    result = model.evaluate(val_dataset)[0]
    acc = model.evaluate(val_dataset)[1]
    
    results.append(result)
    accs.append(acc)
    models.append(model)

    del model, train_dataset, val_dataset

Fold 0
####################
Epoch 1/100
44/44 [==============================] - 1s 30ms/step - loss: 3.2777 - accuracy: 0.4555 - val_loss: 3.3404 - val_accuracy: 0.2888
Epoch 2/100
44/44 [==============================] - 1s 27ms/step - loss: 2.3387 - accuracy: 0.5501 - val_loss: 1.8670 - val_accuracy: 0.5430
Epoch 3/100
44/44 [==============================] - 1s 27ms/step - loss: 2.0642 - accuracy: 0.5814 - val_loss: 1.8103 - val_accuracy: 0.5594
Epoch 4/100
44/44 [==============================] - 1s 26ms/step - loss: 1.8396 - accuracy: 0.6184 - val_loss: 1.6903 - val_accuracy: 0.5708
Epoch 5/100
44/44 [==============================] - 1s 27ms/step - loss: 1.6555 - accuracy: 0.6494 - val_loss: 1.5149 - val_accuracy: 0.6035
Epoch 6/100
44/44 [==============================] - 1s 26ms/step - loss: 1.4933 - accuracy: 0.6789 - val_loss: 1.3473 - val_accuracy: 0.6387
Epoch 7/100
44/44 [==============================] - 1s 27ms/step - loss: 1.3474 - accuracy: 0.7066 - val_loss: 1.2146 -

44/44 [==============================] - 1s 28ms/step - loss: 0.1145 - accuracy: 0.9847 - val_loss: 0.1574 - val_accuracy: 0.9584
Epoch 58/100
44/44 [==============================] - 1s 27ms/step - loss: 0.1063 - accuracy: 0.9868 - val_loss: 0.1605 - val_accuracy: 0.9623
Epoch 59/100
44/44 [==============================] - 1s 27ms/step - loss: 0.1038 - accuracy: 0.9876 - val_loss: 0.1644 - val_accuracy: 0.9619
Epoch 60/100
44/44 [==============================] - 1s 28ms/step - loss: 0.1023 - accuracy: 0.9886 - val_loss: 0.1539 - val_accuracy: 0.9584
Epoch 61/100
44/44 [==============================] - 1s 28ms/step - loss: 0.0963 - accuracy: 0.9883 - val_loss: 0.1537 - val_accuracy: 0.9644
Epoch 62/100
44/44 [==============================] - 1s 26ms/step - loss: 0.1086 - accuracy: 0.9847 - val_loss: 0.2050 - val_accuracy: 0.9367
Epoch 63/100
44/44 [==============================] - 1s 26ms/step - loss: 0.1166 - accuracy: 0.9822 - val_loss: 0.1673 - val_accuracy: 0.9573
Epoch 64/100

44/44 [==============================] - 1s 27ms/step - loss: 1.2329 - accuracy: 0.7312 - val_loss: 1.0794 - val_accuracy: 0.7084
Epoch 9/100
44/44 [==============================] - 1s 27ms/step - loss: 1.1273 - accuracy: 0.7493 - val_loss: 0.9875 - val_accuracy: 0.7397
Epoch 10/100
44/44 [==============================] - 1s 28ms/step - loss: 1.0354 - accuracy: 0.7688 - val_loss: 0.8984 - val_accuracy: 0.7667
Epoch 11/100
44/44 [==============================] - 1s 27ms/step - loss: 0.9595 - accuracy: 0.7817 - val_loss: 0.8255 - val_accuracy: 0.8016
Epoch 12/100
44/44 [==============================] - 1s 28ms/step - loss: 0.8952 - accuracy: 0.7969 - val_loss: 0.7933 - val_accuracy: 0.7923
Epoch 13/100
44/44 [==============================] - 1s 27ms/step - loss: 0.8421 - accuracy: 0.8115 - val_loss: 0.7216 - val_accuracy: 0.8172
Epoch 14/100
44/44 [==============================] - 1s 27ms/step - loss: 0.8009 - accuracy: 0.8169 - val_loss: 0.6631 - val_accuracy: 0.8425
Epoch 15/100


KeyboardInterrupt: 

In [53]:
results

[0.0896417573094368,
 0.15422236919403076,
 0.14694762229919434,
 0.15056133270263672,
 0.15249910950660706,
 0.14939630031585693,
 0.15247102081775665,
 0.08403807878494263,
 0.14435991644859314,
 0.14709262549877167]

In [55]:
accs

[0.9736841917037964,
 0.9573257565498352,
 0.9591038227081299,
 0.9587482213973999,
 0.9576813578605652,
 0.9584074020385742,
 0.9569854140281677,
 0.9797369241714478,
 0.9644507765769958,
 0.9633842706680298]

In [54]:
np.mean(accs)

0.9629508137702942

In [56]:
# 결과 생성
pred_list = []    # 예측 결과를 담을 리스트
for model in models:
  pred = model.predict(series_test)
  pred_list.append(pred)

pred = np.mean(pred_list, axis=0)

In [57]:
# 제출물 생성
sub.iloc[:, 1:] = pred

save_path = os.path.join('sub/test_wt.csv')     # 저장 경로
sub.to_csv(save_path, index=False)